In [1]:
import keras
from keras.models import Model
from keras.layers import Input, Dense, Concatenate, Flatten, Reshape
from keras.layers import GlobalAveragePooling2D, Dropout, UpSampling2D, Conv2D, MaxPooling2D, Activation

import params

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import tqdm
from tqdm import tqdm_notebook as tqdm

Using TensorFlow backend.


In [2]:
descriptor_size = int(params.GetImageSize() / 2 / 2 / 2) * int(params.GetImageSize() / 2 / 2 / 2)

In [3]:
def BuildModel_AE():
    inp = Input(shape=(params.GetImageSize(), params.GetImageSize(), params.GetChannelsNum()))

    conv_1_1 = Conv2D(32, (3, 3), padding='same')(inp)
    conv_1_1 = Activation('relu')(conv_1_1)
    conv_1_2 = Conv2D(32, (3, 3), padding='same')(conv_1_1)
    conv_1_2 = Activation('relu')(conv_1_2)
    pool_1 = MaxPooling2D(2)(conv_1_2)

    conv_2_1 = Conv2D(64, (3, 3), padding='same')(pool_1)
    conv_2_1 = Activation('relu')(conv_2_1)
    conv_2_2 = Conv2D(64, (3, 3), padding='same')(conv_2_1)
    conv_2_2 = Activation('relu')(conv_2_2)
    pool_2 = MaxPooling2D(2)(conv_2_2)

    conv_3_1 = Conv2D(128, (3, 3), padding='same')(pool_2)
    conv_3_1 = Activation('relu')(conv_3_1)
    conv_3_2 = Conv2D(256, (3, 3), padding='same')(conv_3_1)
    conv_3_2 = Activation('relu')(conv_3_2)
    pool_3 = MaxPooling2D(2)(conv_3_2)
    
    flat_1 = Flatten()(pool_3)
    
    descriptor = Dense(descriptor_size, name='descriptor')(flat_1)
    
    descr_resample = Reshape((32, 32, 1))(descriptor)

    up_1 = UpSampling2D(2, interpolation='bilinear')(descr_resample)
    conv_up_1_1 = Conv2D(256, (3, 3), padding='same')(up_1)
    conv_up_1_1 = Activation('relu')(conv_up_1_1)
    conv_up_1_2 = Conv2D(256, (3, 3), padding='same')(conv_up_1_1)
    conv_up_1_2 = Activation('relu')(conv_up_1_2)

    up_2 = UpSampling2D(2, interpolation='bilinear')(conv_up_1_2)
    conv_up_2_1 = Conv2D(128, (3, 3), padding='same')(up_2)
    conv_up_2_1 = Activation('relu')(conv_up_2_1)
    conv_up_2_2 = Conv2D(128, (3, 3), padding='same')(conv_up_2_1)
    conv_up_2_2 = Activation('relu')(conv_up_2_2)

    up_3 = UpSampling2D(2, interpolation='bilinear')(conv_up_2_2)
    conv_up_3_1 = Conv2D(64, (3, 3), padding='same')(up_3)
    conv_up_3_1 = Activation('relu')(conv_up_3_1)
    conv_up_3_2 = Conv2D(3, (3, 3), padding='same')(conv_up_3_1)
    result = Activation('sigmoid')(conv_up_3_2)
    
    return Model(inputs=inp, outputs=result)

best_w = keras.callbacks.ModelCheckpoint('best.h5',
                                monitor='val_loss',
                                verbose=0,
                                save_best_only=True,
                                save_weights_only=False,
                                mode='auto',
                                period=1)

last_w = keras.callbacks.ModelCheckpoint('last.h5',
                                monitor='val_loss',
                                verbose=0,
                                save_best_only=False,
                                save_weights_only=False,
                                mode='auto',
                                period=1)


callbacks = [best_w, last_w]


model = BuildModel_AE()
adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(adam, 'mse')

In [4]:
import cv2
import numpy as np
import os, shutil
import glob
import random

def GenerateImage(image, sample_width, sample_height, use_scaling):
    height, width, _ = image.shape
    
    scale = random.uniform(0.9, 2) if use_scaling else 1
    
    scaled_sample_width = int(round(sample_width))
    scaled_sample_height = int(round(sample_height))
    
    x_offset = random.randint(0, width - scaled_sample_width - 1)
    y_offset = random.randint(0, height - scaled_sample_height - 1)

    sample = image[y_offset:y_offset + scaled_sample_height, x_offset:x_offset + scaled_sample_width]
    sample = cv2.resize(sample, (sample_width, sample_height))
    
    angles = [0, 90, 180, 270]
    angle = random.choice(angles)
    if angle > 0:
        for i in range(int(angle / 90)):
            sample = np.rot90(sample)
    
    return sample

def keras_generator(basepath, batch_size, use_scaling=True, class_template=''):
    origin_img_files = glob.glob(os.path.join(basepath, class_template + '*.*'))
    
    origin_imgs = [ cv2.imread(img_name, cv2.IMREAD_COLOR) for img_name in origin_img_files ]
    
    index = 0
    while True:
        x_batch = []
        y_batch = []
        
        for i in range(batch_size):
            index = random.randint(0, len(origin_imgs) - 1)
            
            img = GenerateImage(origin_imgs[index], params.GetImageSize(), params.GetImageSize(), use_scaling)
            
            x_batch += [img]
            y_batch += [img]
            

        yield np.array(x_batch), np.array(y_batch)

In [5]:
batch_size = 8

In [6]:
history = model.fit_generator(keras_generator('./imgs_classified_reduced', batch_size),
              steps_per_epoch=15,
              epochs=5,
              verbose=1,
              callbacks=callbacks,
              validation_data=keras_generator('./imgs_classified_reduced', batch_size),
              validation_steps=3,
              class_weight=None,
              max_queue_size=10,
              workers=1,
              use_multiprocessing=False,
              shuffle=True,
              initial_epoch=0)

Epoch 1/5
15/15 [==============================] - 298s 20s/step - loss: 7135.3212 - val_loss: 6731.0283


InternalError: Could not allocate ndarray

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from keras.models import load_model
model1 = load_model('best.h5')

In [ ]:
intermediate_layer_model = Model(inputs=model1.input, outputs=model1.get_layer('descriptor').output)

In [ ]:
testing_batch_size = 16

for x, y in keras_generator('./imgs_test', testing_batch_size, False):
    break
intermediate_output = intermediate_layer_model.predict(x)
print(x.shape, intermediate_output.shape)

In [ ]:
import matplotlib.pyplot as plt

data_count = x.shape[0]

fig, axes = plt.subplots(nrows=int(data_count/4), ncols=4, figsize=(20, 20))
for i in range(data_count):
    row = int(i / 4)
    col = int(i % 4)
    axes[row][col].imshow(x[i, :, :, :])
    axes[row][col].set_title('image_' + str(i))
    #axes[i][1].plot(intermediate_output[i,:])


# axes[0][0].imshow(x[0, :, :, :])
# axes[1][0].imshow(x[1, :, :, :])
# axes[2][0].imshow(x[2, :, :, :])
# axes[3][0].imshow(x[3, :, :, :])

# #plt.plot(t, t, 'r--', t, t**2, 'bs', t, t**3, 'g^')
# axes[0][1].plot(intermediate_output[0,:], 'r')
# axes[1][1].plot(intermediate_output[1,:], 'b')
# axes[2][1].plot(intermediate_output[2,:], 'g')
# axes[3][1].plot(intermediate_output[3,:], 'k')

plt.show()

In [ ]:
in_data = intermediate_output.transpose()
mean, eigenvectors = cv2.PCACompute(in_data, np.mean(in_data, axis=0).reshape(1,-1), maxComponents=2)

In [ ]:
mean, eigenvectors

In [ ]:
eigenvectors[:,0]
view_data = eigenvectors

fig, ax = plt.subplots()
ax.scatter(view_data[0,:], view_data[1,:])

for i in range(len(view_data[0,:])):
    ax.annotate('  ' + str(i), (view_data[0,:][i], view_data[1,:][i]))

plt.show()

In [ ]:
testing_batch_size = 16

classes_num = 3
classified_data = np.zeros((testing_batch_size * classes_num, descriptor_size)).astype(np.float32)

samples = []

for class_idx in range(classes_num):
    for x, _ in keras_generator('./imgs_classified_reduced', testing_batch_size, False, str(class_idx)):
        break
    sp = testing_batch_size * class_idx
    tp = testing_batch_size * (class_idx + 1)
    classified_data[sp:tp] = intermediate_layer_model.predict(x)
    samples.append(x[:, :, :, :])


In [ ]:
data_count = classified_data.shape[0]

fig, axes = plt.subplots(nrows=int(classes_num), ncols=4, figsize=(20, 20))
for row in range(classes_num):
    for col in range(4):
        axes[row][col].imshow(samples[row][col, :, :, :])
    
plt.show()

In [ ]:
in_data = classified_data.transpose()
mean, eigenvectors = cv2.PCACompute(in_data, np.mean(in_data, axis=0).reshape(1,-1), maxComponents=2)


In [ ]:
eigenvectors.shape

In [ ]:
view_data = eigenvectors

fig, ax = plt.subplots(figsize=(16, 12), dpi=80)

colors = ['red', 'green', 'blue', 'magenta', 'black', 'cyan', 'yellow', 'gray', 'brown', 'orange']
for class_idx in range(classes_num):
    sp = testing_batch_size * class_idx
    tp = testing_batch_size * (class_idx + 1)
    ax.scatter(view_data[0,sp:tp], view_data[1,sp:tp], c=colors[class_idx])

plt.show()

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

view_data = eigenvectors

fig, ax = plt.subplots(figsize=(32, 24), dpi=80)

# colors = ['red', 'green', 'blue', 'magenta', 'black']
# for class_idx in range(classes_num):
#     sp = testing_batch_size * class_idx
#     tp = testing_batch_size * (class_idx + 1)
#     ax.scatter(view_data[0,sp:tp], view_data[1,sp:tp], c=colors[class_idx])
    
zoom = 0.3
for row in range(classes_num):
    for col in range(testing_batch_size):
        im = OffsetImage(samples[row][col, :, :, :], zoom=zoom)
        sp = testing_batch_size * row + col
        ab = AnnotationBbox(im, (view_data[0,sp], view_data[1,sp]), xycoords='data', frameon=False)
        ax.add_artist(ab)
ax.update_datalim(np.column_stack([view_data[0,:], view_data[1,:]]))
ax.autoscale()

plt.show()